## 1. keras-Applications
- https://keras.io/api/applications/
- 선행학습된 가중치와 함께 사용할 수 있도록 제공되는 딥러닝 모델

> DB-Engines Ranking
> https://db-engines.com/en/ranking

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.applications import EfficientNetB0    # 위의 사이트에 쓰여 있는 모델이름 import

import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
# 모델을 얻어오는 함수
# imagenet : 특정 사진을 보여줬을 때 이 사진이 무엇인지 맞출 수 있는 프로젝트에서 시작된 모델
# include_top : 상단에 fully connected 계층을 포함할지 여부

def get_model(input_shape):
    inputs = keras.Input(input_shape)  # input layer 생성
    base_model = EfficientNetB0(
        input_shape = input_shape,
        weights = 'imagenet',
        include_top = False,
        pooling = 'avg'    # pooling 기억안나면 5_모델구현 다시 보기
    )
    
    x = base_model(inputs)
    output = layers.Dense(1, activation='sigmoid')(x)    # 이진 값으로 리턴
    model = keras.Model(inputs, output)
    return model

In [3]:
input_shape = (256, 256, 3)
model = get_model(input_shape)

In [4]:
adam = keras.optimizers.Adam(lr=0.0001)

c:\program files\python37\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [5]:
model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics='accuracy'
)

In [6]:
model.summary()

# 파라미터(params) 증가 <= 7_Albumentations 라이브러리와 비교

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________


In [10]:
# 7_Albunmentation

import albumentations as A

class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            self.transform = A.Compose([
                # 수평
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=15
                ),
                # 구멍을 최대 8개 Dropout
                A.CoarseDropout(
                    p=0.5,
                    max_holes=8,
                    max_height=int(0.1 * size),
                    max_width=int(0.1 * size)
                ),
                A.RandomBrightnessContrast(p=0.2)
            ])
        
    # **kwargs : 매개변수 개수 상관없이 받아서 리스트 처리
    def __call__(self, **kwargs):
        if self.transform :
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img
            
        

In [11]:
import math

class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, fold, image_size, mode='train',  shuffle=True):
        self.batch_size = batch_size
        self.fold = fold
        self.image_size = image_size
        self.mode = mode
        self.shuffle = shuffle
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
            
            
        # https://github.com/tensorflow/models/issues/3134
        # 파일 이슈 -> 삭제
        # Egyptian_Mau_14, 139, 145, 156, 167, 177, 186, 191;
        # Abyssinian_5, 34. 121, 116
        
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'Abyssinian_121',
            'Abyssinian_116'
        ]
        
        # isin : 포함을 의미
        self.df = self.df[~self.df['file_name'].isin(invalid_filenames)]    # invalid_filenames을 file_name으로 찾아서 그것을 제외한 데이터프레임 생성
        self.transform = Augmentation(image_size, mode)
        
        self.on_epoch_end()
        
    # sample() : 전체 row에서 몇 %의 데이터를 return 할 것인지 설정
    # frac : 데이터프레임으로 부터 특정 비율로 무작위 표본 추출을 하고 싶으면 0~1 사이의 부동소수점 입력
    # reset_index() : drop=True 옵션을 설정하면 인덱스 열을 보통의 자료열로 설정하는 것이 아니라 버림
    # 여기서 인데스 열이란 데이터프레임 맨 앞 열에 생성되는 인덱스 열을 말함
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
            
    # len() 함수를 재구현  (공유파일 첫 달에서 스페셜매소드 파일에서 다뤘음)
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
    
    # 배열을 쓸 때 자동으로 불러지는 것
    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = (idx + 1) * self.batch_size
        data = self.df.iloc[start:end]
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        # iterrows
        # 데이터프레임에서 row의 개수만큼 반복
        # 튜플로 리턴[index, 행정보(Series)]
        for _, r in data.iterrows():             
            file_name = r['file_name']
            image = cv2.imread(f'D:/data_ai/5_AI/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.image_size, self.image_size))
            
            if self.mode == 'train':
                image = image.astype('uint8')
                image = self.transform(image=image)
                
            image = image.astype('float32')
            image = image / 255    # 정규화
            
            label = int(r['species']) - 1    # 밑에서 class_name 인덱스와 맞추기위해 (-1)해줌
            
            batch_x.append(image)
            batch_y.append(label)
            
        return batch_x, batch_y

In [12]:
csv_path = 'D:/data_ai/5_AI/kfolds.csv'

train_generator = DataGenerator(
    # 옵션
    batch_size = 128, 
    csv_path = csv_path, 
    fold = 1, 
    image_size = 256, 
    mode = 'train', 
    shuffle = True
)

valid_generator = DataGenerator(
    # 옵션
    batch_size = 128, 
    csv_path = csv_path, 
    fold = 1, 
    image_size = 256, 
    mode = 'val', 
    shuffle = True
)

In [13]:
history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs=10,
    verbose=1
)

# ResourceExhaustedError

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'model/efficientnetb0/block5b_expand_conv/Conv2D' defined at (most recent call last):
    File "c:\program files\python37\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\program files\python37\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\program files\python37\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\program files\python37\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\program files\python37\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\program files\python37\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\program files\python37\lib\asyncio\base_events.py", line 541, in run_forever
      self._